In [1]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import tensorflow as tf
import io

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [3]:
dataset = load_dataset('mt_eng_vietnamese', 'iwslt2015-vi-en')

Found cached dataset mt_eng_vietnamese (C:/Users/PC/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Tokenize dataset
def tokenize_function(examples):
    vietnamese_sentences = []
    english_sentences = []
    for _ in examples["translation"]:
        vietnamese_sentences.append(_["vi"])
        english_sentences.append(_["en"])
    #vietnamese_sentences = examples["translation"]["vi"]
    #english_sentences = examples["translation"]["en"]
    
    tokenized_inputs = tokenizer(vietnamese_sentences, truncation=True, padding="max_length")
    tokenized_targets = tokenizer(english_sentences, truncation=True, padding="max_length")
    
    examples["input_ids"] = tokenized_inputs.input_ids
    examples["attention_mask"] = tokenized_inputs.attention_mask
    examples["labels"] = tokenized_targets.input_ids
    
    return examples

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split dataset into train and validation sets
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]

Loading cached processed dataset at C:\Users\PC\.cache\huggingface\datasets\mt_eng_vietnamese\iwslt2015-vi-en\1.0.0\53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71\cache-37e88c94a6042c0f.arrow
Loading cached processed dataset at C:\Users\PC\.cache\huggingface\datasets\mt_eng_vietnamese\iwslt2015-vi-en\1.0.0\53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71\cache-b26be17c17c26bfa.arrow
Loading cached processed dataset at C:\Users\PC\.cache\huggingface\datasets\mt_eng_vietnamese\iwslt2015-vi-en\1.0.0\53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71\cache-7dac638c6c0744bf.arrow


In [5]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [7]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt_eng_vietnamese_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=50,
    eval_steps=100,
    logging_steps=100,
    overwrite_output_dir=True,
    save_total_limit=3,
)


In [8]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()


c:\Users\PC\miniconda3\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/99990 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 252.00 MiB (GPU 0; 4.00 GiB total capacity; 3.23 GiB already allocated; 0 bytes free; 3.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF